In [1]:
import pandas as pd

In [2]:
path = '210929_tess_5min_24h'

In [3]:
hce_bids = pd.read_csv(path + '/hce_bids.csv', index_col=0, parse_dates=True)
hce_bids['start_time'].iloc[0]

'2021-09-22 00:00:00'

In [21]:
transformer_interval = pd.read_csv(path + '/transformer_intervals.csv', index_col=0, parse_dates=True)
transformer_interval

,transformer_id,import_capacity,export_capacity,q,unresp_load,start_time,end_time
transformer_interval_id,,,,,,,
1,1,951.701,475.851,0.000,0.000,2021-09-22 00:00:00,2021-09-22 00:05:00
2,1,984.151,492.075,4.015,4.015,2021-09-22 00:05:00,2021-09-22 00:10:00
3,1,919.672,459.836,3.912,3.912,2021-09-22 00:10:00,2021-09-22 00:15:00
4,1,928.804,464.402,3.912,3.912,2021-09-22 00:15:00,2021-09-22 00:20:00
5,1,1090.370,545.186,3.912,3.912,2021-09-22 00:20:00,2021-09-22 00:25:00
...,...,...,...,...,...,...,...
283,1,957.365,478.683,4.434,-5995.570,2021-09-22 23:40:00,2021-09-22 23:45:00
284,1,1019.960,509.982,4.434,4.434,2021-09-22 23:45:00,2021-09-22 23:50:00
285,1,1054.670,527.335,4.434,4.434,2021-09-22 23:50:00,2021-09-22 23:55:00


In [18]:
meter_interval = pd.read_csv(path + '/meter_intervals.csv', index_col=0, parse_dates=True)

In [19]:
market = pd.read_csv(path + '/markets.csv', index_col=0, parse_dates=True).iloc[:1]
market_interval = pd.read_csv(path + '/market_intervals.csv', index_col=0, parse_dates=True)

In [20]:
p_clear = list(market_interval["p_clear"])

## Choose period for which cash flow should be calculated

In [8]:
market_id = 1
start_time = '2021-09-22 12:00:00'

## 1 ) Payments from retail income

In [35]:
RR = 0.20 # hard-coded, need to check
interval = market.loc[market_id]['ts']
unresp_load = transformer_interval['unresp_load']
transformer_interval["retail_income"] = RR*unresp_load*interval/3600. # price*energy
transformer_interval

,transformer_id,import_capacity,export_capacity,q,unresp_load,start_time,end_time,retail_income
transformer_interval_id,,,,,,,,
1,1,951.701,475.851,0.000,0.000,2021-09-22 00:00:00,2021-09-22 00:05:00,0.000000
2,1,984.151,492.075,4.015,4.015,2021-09-22 00:05:00,2021-09-22 00:10:00,0.066917
3,1,919.672,459.836,3.912,3.912,2021-09-22 00:10:00,2021-09-22 00:15:00,0.065200
4,1,928.804,464.402,3.912,3.912,2021-09-22 00:15:00,2021-09-22 00:20:00,0.065200
5,1,1090.370,545.186,3.912,3.912,2021-09-22 00:20:00,2021-09-22 00:25:00,0.065200
...,...,...,...,...,...,...,...,...
283,1,957.365,478.683,4.434,-5995.570,2021-09-22 23:40:00,2021-09-22 23:45:00,-99.926167
284,1,1019.960,509.982,4.434,4.434,2021-09-22 23:45:00,2021-09-22 23:50:00,0.073900
285,1,1054.670,527.335,4.434,4.434,2021-09-22 23:50:00,2021-09-22 23:55:00,0.073900


## 2) Payments to WS supply

In [79]:
import_power = transformer_interval['q']
mc_WS = hce_bids['p_bid'].loc[(hce_bids['start_time'] == start_time) & (hce_bids['comment'] == 'import')].iloc[-1]
transformer_interval["import_expenses"] = mc_WS*import_power*interval/3600. # price*energy

## 3) PAYMENTS TO PV OWNERS: Sum all ((qmtp in TABLE meter_intervals for which p_bid <= p_clear))*p_clear

In [70]:
bids = meter_interval
bids

,meter_id,rate_id,start_time,end_time,e,qmtp,p_bid,q_bid,is_bid,mode_dispatch,mode_market
meter_interval_id,,,,,,,,,,,
1,1,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1
2,2,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1
3,3,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1
4,4,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1
5,5,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1722,2,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1
1723,3,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1
1724,4,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1


In [58]:
# Get clearing price for each bid for later comparison at time t
bids_interval = bids.merge(market_interval, how="left")
assert len(bids) == len(bids_interval) # should not increase no of rows
bids_interval

,meter_id,rate_id,start_time,end_time,e,qmtp,p_bid,q_bid,is_bid,mode_dispatch,mode_market,market_id,p_exp,p_dev,p_clear,q_clear,alpha
0,1,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0
1,2,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0
2,3,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0
3,4,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0
4,5,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1721,2,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,0.468286,0.075985,0.80,4.434,1.0
1722,3,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,0.468286,0.075985,0.80,4.434,1.0
1723,4,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,0.468286,0.075985,0.80,4.434,1.0
1724,5,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,0.468286,0.075985,0.80,4.434,1.0


In [59]:
cleared_bids = bids_interval.loc[bids_interval.p_bid <= bids_interval.p_clear]
cleared_bids

,meter_id,rate_id,start_time,end_time,e,qmtp,p_bid,q_bid,is_bid,mode_dispatch,mode_market,market_id,p_exp,p_dev,p_clear,q_clear,alpha
0,1,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0
1,2,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0
2,3,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0
3,4,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0
4,5,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1721,2,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,0.468286,0.075985,0.80,4.434,1.0
1722,3,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,0.468286,0.075985,0.80,4.434,1.0
1723,4,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,0.468286,0.075985,0.80,4.434,1.0
1724,5,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,0.468286,0.075985,0.80,4.434,1.0


In [74]:
cleared_bids["payments_to_pv_owner"] = cleared_bids["p_clear"]*cleared_bids["q_bid"]*interval/3600.
cleared_bids

/var/folders/tp/xzkt258d77dgg1hgjm1bkb6x_vqyb8/T/ipykernel_78161/811425598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleared_bids["payments_to_pv_owner"] = cleared_bids["p_clear"]*cleared_bids["q_bid"]*interval/3600.


,meter_id,rate_id,start_time,end_time,e,qmtp,p_bid,q_bid,is_bid,mode_dispatch,mode_market,market_id,p_exp,p_dev,p_clear,q_clear,alpha,payments_to_pv_owner
0,1,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0,0.0
1,2,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0,0.0
2,3,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0,0.0
3,4,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0,0.0
4,5,1,2021-09-22 00:00:00,2021-09-22 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,50.000000,1.000000,0.81,0.000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1721,2,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,0.468286,0.075985,0.80,4.434,1.0,0.0
1722,3,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,0.468286,0.075985,0.80,4.434,1.0,0.0
1723,4,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,0.468286,0.075985,0.80,4.434,1.0,0.0
1724,5,1,2021-09-23 00:00:00,2021-09-23 00:05:00,0.0,0.0,0,0.0,1,1,1,1.0,0.468286,0.075985,0.80,4.434,1.0,0.0


In [72]:
payments_to_pv_owners = cleared_bids["payments_to_pv_owner"].sum()
payments_to_pv_owners

12.686584058722165

## 4) retail income MINUS import costs MINUS payments to PV owners → THIS IS WHAT GETS PLOTTED



In [76]:
transformer_interval["retail_income"] - transformer_interval["import_expenses"] - payments_to_pv_owners # cash flow / net revenue at time t

transformer_interval_id
1      -12.686584
2      -12.800130
3      -12.797217
4      -12.797217
5      -12.797217
          ...    
283   -112.812046
284    -12.811979
285    -12.811979
286    -12.811979
287    -12.811979
Length: 287, dtype: float64

In [85]:
def func(start_time, market_id=1):
    RR = 0.20 # hard-coded, need to check
    interval = market.loc[market_id]['ts']
    unresp_load = transformer_interval['unresp_load'].loc[transformer_interval['start_time'] == start_time].iloc[-1]
    retail_income = RR*unresp_load*interval/3600. # price*energy
    import_power = transformer_interval['q'].loc[transformer_interval['start_time'] == start_time].iloc[-1]
    mc_WS = hce_bids['p_bid'].loc[(hce_bids['start_time'] == start_time) & (hce_bids['comment'] == 'import')].iloc[-1]
    import_expenses = mc_WS*import_power*interval/3600. # price*energy
    bids = meter_interval.loc[meter_interval['start_time'] == start_time]
    bids_interval = bids.merge(market_interval, on="start_time", how="left")
    assert len(bids) == len(bids_interval) # should not increase no of rows
    cleared_bids["payments_to_pv_owner"] = cleared_bids["p_clear"]*cleared_bids["q_bid"]*interval/3600.
    payments_to_pv_owners = cleared_bids["payments_to_pv_owner"].sum()
    return retail_income - import_expenses - payments_to_pv_owners # cash flow / net revenue at time t

In [86]:
func('2021-09-22 12:00:00')

/var/folders/tp/xzkt258d77dgg1hgjm1bkb6x_vqyb8/T/ipykernel_78161/2834325105.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleared_bids["payments_to_pv_owner"] = cleared_bids["p_clear"]*cleared_bids["q_bid"]*interval/3600.


-12.823875001638832